# Classification models for pose prediction

In [1]:
import os
import numpy as np
import pandas as pd
import oddt
import oddt.pandas as opd
from oddt.pandas import ChemDataFrame
from oddt.fingerprints import PLEC
from scipy import stats
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import matthews_corrcoef, precision_recall_curve, accuracy_score, auc
from sklearn.model_selection import cross_val_predict
from sklearn.neural_network import MLPClassifier
from sklearn.utils import parallel_backend
from xgboost.sklearn import XGBClassifier
from joblib import Parallel, delayed
from tqdm import tqdm
import glob
import tempfile
import hyperopt
from hyperopt import hp, tpe, Trials, fmin, STATUS_OK, space_eval

In [2]:
#Load csv files containing training and test data

train_data = pd.read_csv("/home/vktrannguyen/Projects/redocked/PLEC/data_partitions/training_1/RMSD/training_data_poses.csv")
Train_Class = train_data['Classification']
test_data = pd.read_csv("/home/vktrannguyen/Projects/redocked/PLEC/hard_test/RMSD/hard_test_data_poses.csv")
Test_Class = test_data['Classification']

In [3]:
#Load PLEC fingerprints of training and test data

d_train_csv = pd.read_csv('/home/vktrannguyen/Projects/redocked/PLEC/data_partitions/training_1/PLEC/training_data_PLEC.csv', header=None)
d_test_csv = pd.read_csv('/home/vktrannguyen/Projects/redocked/PLEC/hard_test/PLEC/hard_test_data_PLEC.csv', header=None)

## RF models 

#### Without hyperparameter tuning 

In [8]:
#Train the RF model on the training molecules:
rf_plec = RandomForestClassifier(n_estimators = 400, max_features = 'sqrt', n_jobs = 30)
rf_plec.fit(d_train_csv, Train_Class)

#Test the RF model on the test molecules:
prediction_test_rf_plec_class = rf_plec.predict(d_test_csv)
prediction_test_rf_plec_prob = rf_plec.predict_proba(d_test_csv)

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_rf = pd.DataFrame({"Good_Pose_Prob": prediction_test_rf_plec_prob[:, 1],
                               "Predicted_Class": prediction_test_rf_plec_class,
                               "Real_Class": Test_Class})

rmsd = test_data.iloc[:, 1]
pose = test_data.iloc[:, 0]

plec_result_rf['RMSD'] = rmsd
plec_result_rf['Pose'] = pose

plec_result_rf.to_csv("/home/vktrannguyen/Projects/redocked/PLEC/hard_test/results/RF/RF_no-tuning_05.csv")

#### With hyperparameter tuning 

In [13]:
#Define the search space for optimal parameters:
space = {"n_estimators": hp.choice("n_estimators", np.arange(100, 500, 5000)),
         "max_depth": hp.choice("max_depth", [1, 2, 3, 4, 5]),
         "criterion": hp.choice("criterion", ['gini', 'entropy'])}

#Define the function for hyperparameter tuning:
def hyperparameter_tuning_randomforest(space):
    model = RandomForestClassifier(n_estimators = int(space['n_estimators']),
                                   max_depth = int(space['max_depth']),
                                   criterion = space['criterion'], n_jobs = 40)
    model.fit(np.array(d_train_csv), Train_Class)
    predicted_train = model.predict(np.array(d_train_csv))
    mcc = matthews_corrcoef(Train_Class, predicted_train)
    return {'loss': 1-mcc, 'status': STATUS_OK, 'model': model}
    
#Search for optimal parameters:
trials = Trials()
best_rf_classification = fmin(fn = hyperparameter_tuning_randomforest, space = space, algo = tpe.suggest,
                              max_evals = 10, trials = trials)
best_params = space_eval(space, best_rf_classification)

#Optimal parameters:
best_params

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:18<00:00,  1.88s/trial, best loss: 0.7778339991353744]


{'criterion': 'entropy', 'max_depth': 5, 'n_estimators': 100}

In [14]:
#Train the RF model on the training molecules, using optimal parameters:
rf_plec = RandomForestClassifier(n_estimators = best_params['n_estimators'], 
                                 max_depth = best_params['max_depth'], 
                                 criterion = best_params['criterion'],
                                 max_features = 'sqrt', n_jobs = 30)
rf_plec.fit(d_train_csv, Train_Class)

#Test the RF model on the test molecules:
prediction_test_rf_plec_class = rf_plec.predict(d_test_csv)
prediction_test_rf_plec_prob = rf_plec.predict_proba(d_test_csv)

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_rf = pd.DataFrame({"Good_Pose_Prob": prediction_test_rf_plec_prob[:, 1],
                               "Predicted_Class": prediction_test_rf_plec_class,
                               "Real_Class": Test_Class})

rmsd = test_data.iloc[:, 1]
pose = test_data.iloc[:, 0]

plec_result_rf['RMSD'] = rmsd
plec_result_rf['Pose'] = pose

plec_result_rf.to_csv("/home/vktrannguyen/Projects/redocked/PLEC/data_partitions/results_1/RF/RF_tuning_05.csv")

## XGB models 

#### Without hyperparameter tuning 

In [13]:
#Train the XGB model on the training molecules:
xgb_plec = XGBClassifier(n_jobs = 40)
xgb_plec.fit(np.array(d_train_csv), Train_Class)

#Test the XGB model on the test molecules:
prediction_test_xgb_plec_class = xgb_plec.predict(np.array(d_test_csv))
prediction_test_xgb_plec_prob = xgb_plec.predict_proba(np.array(d_test_csv))

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_xgb = pd.DataFrame({"Good_Pose_Prob": prediction_test_xgb_plec_prob[:, 1],
                                "Predicted_Class": prediction_test_xgb_plec_class,
                                "Real_Class": Test_Class})

rmsd = test_data.iloc[:, 1]
pose = test_data.iloc[:, 0]

plec_result_xgb['RMSD'] = rmsd
plec_result_xgb['Pose'] = pose

plec_result_xgb.to_csv("/home/vktrannguyen/Projects/redocked/PLEC/hard_test/results/XGB/XGB_no-tuning_05.csv")

[17:43:16] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


#### With hyperparameter tuning 

In [9]:
#Define the search space for optimal parameters:
space = {"max_depth": hp.choice("max_depth", [3, 4, 5, 6, 7, 8, 9, 10, 18]),
         "reg_lambda": hp.uniform("reg_lambda", 0, 1), 
         "n_estimators": hp.uniform("n_estimators", 1000, 5000)} 

#Define the function for hyperparameter tuning:
def hyperparameter_tuning_XGB(space):
    model = XGBClassifier(objective = "binary:logistic", 
                          max_depth = space['max_depth'],
                          reg_lambda = space['reg_lambda'],
                          n_estimators = int(space['n_estimators']))
    model.fit(np.array(d_train_csv), Train_Class)
    predicted_train = model.predict(np.array(d_train_csv))
    mcc = matthews_corrcoef(Train_Class, predicted_train)
    return {'loss': 1-mcc, 'status': STATUS_OK, 'model': model}
    
#Search for optimal parameters:
trials = Trials()
best_xgb_classification = fmin(fn = hyperparameter_tuning_XGB, space = space, algo = tpe.suggest, 
                               max_evals = 10, trials = trials)
best_params = space_eval(space, best_xgb_classification)

#Optimal parameters:
best_params

  0%|                                                                                                                                                                | 0/10 [00:00<?, ?trial/s, best loss=?]

/home/vktrannguyen/anaconda3/envs/code-env/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



[17:13:33] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
 10%|████████████▋                                                                                                                  | 1/10 [07:01<1:03:16, 421.83s/trial, best loss: 0.00019305715655837385]

/home/vktrannguyen/anaconda3/envs/code-env/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



[17:20:34] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
 20%|█████████████████████████▍                                                                                                     | 2/10 [17:55<1:14:27, 558.43s/trial, best loss: 0.00019305715655837385]

...

/home/vktrannguyen/anaconda3/envs/code-env/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



[18:36:52] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
 90%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎            | 9/10 [1:38:26<12:13, 733.53s/trial, best loss: 0.00019305715655837385]

/home/vktrannguyen/anaconda3/envs/code-env/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



[18:51:59] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [1:47:58<00:00, 647.85s/trial, best loss: 0.00019305715655837385]


{'max_depth': 10,
 'n_estimators': 1778.2134802018963,
 'reg_lambda': 0.2640422650249682}

In [25]:
#Train the XGB model on the training molecules, using optimal parameters:
xgb_plec = XGBClassifier(objective = "binary:logistic", 
                         max_depth = best_params['max_depth'], 
                         reg_lambda = best_params['reg_lambda'], 
                         n_estimators = int(best_params['n_estimators']),
                         n_jobs = 40, random_state = 0)
xgb_plec.fit(np.array(d_train_csv), Train_Class)

#Test the XGB model on the test molecules:
prediction_test_xgb_plec_class = xgb_plec.predict(np.array(d_test_csv))
prediction_test_xgb_plec_prob = xgb_plec.predict_proba(np.array(d_test_csv))

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_xgb = pd.DataFrame({"Good_Pose_Prob": prediction_test_xgb_plec_prob[:, 1],
                                "Predicted_Class": prediction_test_xgb_plec_class,
                                "Real_Class": Test_Class})

rmsd = test_data.iloc[:, 1]
pose = test_data.iloc[:, 0]

plec_result_xgb['RMSD'] = rmsd
plec_result_xgb['Pose'] = pose

plec_result_xgb.to_csv("/home/vktrannguyen/Projects/redocked/PLEC/data_partitions/results_1/XGB/XGB_tuning_05.csv")

[10:38:30] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


## SVM models

#### Without hyperparameter tuning 

In [19]:
#Train the SVM model on the training molecules:
svm_plec = SVC(degree = 3, kernel = "rbf", probability = True)
svm_plec.fit(d_train_csv, Train_Class)

#Test the SVM model on the test molecules:
prediction_test_svm_plec_class = svm_plec.predict(d_test_csv)
prediction_test_svm_plec_prob = svm_plec.predict_proba(d_test_csv)

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_svm  = pd.DataFrame({"Good_Pose_Prob": prediction_test_svm_plec_prob[:, 1],
                                 "Predicted_Class": prediction_test_svm_plec_class,
                                 "Real_Class": Test_Class})

rmsd = test_data.iloc[:, 1]
pose = test_data.iloc[:, 0]

plec_result_svm['RMSD'] = rmsd
plec_result_svm['Pose'] = pose

plec_result_svm.to_csv("/home/vktrannguyen/Projects/redocked/PLEC/hard_test/results/SVM/SVM_no-tuning_01.csv")

In [20]:
#Train the SVM model on the training molecules:
svm_plec = SVC(degree = 3, kernel = "rbf", probability = True)
svm_plec.fit(d_train_csv, Train_Class)

#Test the SVM model on the test molecules:
prediction_test_svm_plec_class = svm_plec.predict(d_test_csv)
prediction_test_svm_plec_prob = svm_plec.predict_proba(d_test_csv)

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_svm  = pd.DataFrame({"Good_Pose_Prob": prediction_test_svm_plec_prob[:, 1],
                                 "Predicted_Class": prediction_test_svm_plec_class,
                                 "Real_Class": Test_Class})

rmsd = test_data.iloc[:, 1]
pose = test_data.iloc[:, 0]

plec_result_svm['RMSD'] = rmsd
plec_result_svm['Pose'] = pose

plec_result_svm.to_csv("/home/vktrannguyen/Projects/redocked/PLEC/hard_test/results/SVM/SVM_no-tuning_02.csv")

In [21]:
#Train the SVM model on the training molecules:
svm_plec = SVC(degree = 3, kernel = "rbf", probability = True)
svm_plec.fit(d_train_csv, Train_Class)

#Test the SVM model on the test molecules:
prediction_test_svm_plec_class = svm_plec.predict(d_test_csv)
prediction_test_svm_plec_prob = svm_plec.predict_proba(d_test_csv)

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_svm  = pd.DataFrame({"Good_Pose_Prob": prediction_test_svm_plec_prob[:, 1],
                                 "Predicted_Class": prediction_test_svm_plec_class,
                                 "Real_Class": Test_Class})

rmsd = test_data.iloc[:, 1]
pose = test_data.iloc[:, 0]

plec_result_svm['RMSD'] = rmsd
plec_result_svm['Pose'] = pose

plec_result_svm.to_csv("/home/vktrannguyen/Projects/redocked/PLEC/hard_test/results/SVM/SVM_no-tuning_03.csv")

In [22]:
#Train the SVM model on the training molecules:
svm_plec = SVC(degree = 3, kernel = "rbf", probability = True)
svm_plec.fit(d_train_csv, Train_Class)

#Test the SVM model on the test molecules:
prediction_test_svm_plec_class = svm_plec.predict(d_test_csv)
prediction_test_svm_plec_prob = svm_plec.predict_proba(d_test_csv)

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_svm  = pd.DataFrame({"Good_Pose_Prob": prediction_test_svm_plec_prob[:, 1],
                                 "Predicted_Class": prediction_test_svm_plec_class,
                                 "Real_Class": Test_Class})

rmsd = test_data.iloc[:, 1]
pose = test_data.iloc[:, 0]

plec_result_svm['RMSD'] = rmsd
plec_result_svm['Pose'] = pose

plec_result_svm.to_csv("/home/vktrannguyen/Projects/redocked/PLEC/hard_test/results/SVM/SVM_no-tuning_04.csv")

In [23]:
#Train the SVM model on the training molecules:
svm_plec = SVC(degree = 3, kernel = "rbf", probability = True)
svm_plec.fit(d_train_csv, Train_Class)

#Test the SVM model on the test molecules:
prediction_test_svm_plec_class = svm_plec.predict(d_test_csv)
prediction_test_svm_plec_prob = svm_plec.predict_proba(d_test_csv)

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_svm  = pd.DataFrame({"Good_Pose_Prob": prediction_test_svm_plec_prob[:, 1],
                                 "Predicted_Class": prediction_test_svm_plec_class,
                                 "Real_Class": Test_Class})

rmsd = test_data.iloc[:, 1]
pose = test_data.iloc[:, 0]

plec_result_svm['RMSD'] = rmsd
plec_result_svm['Pose'] = pose

plec_result_svm.to_csv("/home/vktrannguyen/Projects/redocked/PLEC/hard_test/results/SVM/SVM_no-tuning_05.csv")

## ANN models 

#### Without hyperparameter tuning 

In [14]:
#Train the ANN model on the training molecules:
ann_plec = MLPClassifier(max_iter = 9000)
ann_plec.fit(d_train_csv, Train_Class)

#Test the ANN model on the test molecules:
prediction_test_ann_plec_class = ann_plec.predict(d_test_csv)
prediction_test_ann_plec_prob = ann_plec.predict_proba(d_test_csv)

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_ann = pd.DataFrame({"Good_Pose_Prob": prediction_test_ann_plec_prob[:, 1],
                                "Predicted_Class": prediction_test_ann_plec_class,
                                "Real_Class": Test_Class})

rmsd = test_data.iloc[:, 1]
pose = test_data.iloc[:, 0]

plec_result_ann['RMSD'] = rmsd
plec_result_ann['Pose'] = pose

plec_result_ann.to_csv("/home/vktrannguyen/Projects/redocked/PLEC/hard_test/results/ANN/ANN_no-tuning_01.csv")

In [15]:
#Train the ANN model on the training molecules:
ann_plec = MLPClassifier(max_iter = 9000)
ann_plec.fit(d_train_csv, Train_Class)

#Test the ANN model on the test molecules:
prediction_test_ann_plec_class = ann_plec.predict(d_test_csv)
prediction_test_ann_plec_prob = ann_plec.predict_proba(d_test_csv)

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_ann = pd.DataFrame({"Good_Pose_Prob": prediction_test_ann_plec_prob[:, 1],
                                "Predicted_Class": prediction_test_ann_plec_class,
                                "Real_Class": Test_Class})

rmsd = test_data.iloc[:, 1]
pose = test_data.iloc[:, 0]

plec_result_ann['RMSD'] = rmsd
plec_result_ann['Pose'] = pose

plec_result_ann.to_csv("/home/vktrannguyen/Projects/redocked/PLEC/hard_test/results/ANN/ANN_no-tuning_02.csv")

In [16]:
#Train the ANN model on the training molecules:
ann_plec = MLPClassifier(max_iter = 9000)
ann_plec.fit(d_train_csv, Train_Class)

#Test the ANN model on the test molecules:
prediction_test_ann_plec_class = ann_plec.predict(d_test_csv)
prediction_test_ann_plec_prob = ann_plec.predict_proba(d_test_csv)

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_ann = pd.DataFrame({"Good_Pose_Prob": prediction_test_ann_plec_prob[:, 1],
                                "Predicted_Class": prediction_test_ann_plec_class,
                                "Real_Class": Test_Class})

rmsd = test_data.iloc[:, 1]
pose = test_data.iloc[:, 0]

plec_result_ann['RMSD'] = rmsd
plec_result_ann['Pose'] = pose

plec_result_ann.to_csv("/home/vktrannguyen/Projects/redocked/PLEC/hard_test/results/ANN/ANN_no-tuning_03.csv")

In [17]:
#Train the ANN model on the training molecules:
ann_plec = MLPClassifier(max_iter = 9000)
ann_plec.fit(d_train_csv, Train_Class)

#Test the ANN model on the test molecules:
prediction_test_ann_plec_class = ann_plec.predict(d_test_csv)
prediction_test_ann_plec_prob = ann_plec.predict_proba(d_test_csv)

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_ann = pd.DataFrame({"Good_Pose_Prob": prediction_test_ann_plec_prob[:, 1],
                                "Predicted_Class": prediction_test_ann_plec_class,
                                "Real_Class": Test_Class})

rmsd = test_data.iloc[:, 1]
pose = test_data.iloc[:, 0]

plec_result_ann['RMSD'] = rmsd
plec_result_ann['Pose'] = pose

plec_result_ann.to_csv("/home/vktrannguyen/Projects/redocked/PLEC/hard_test/results/ANN/ANN_no-tuning_04.csv")

In [18]:
#Train the ANN model on the training molecules:
ann_plec = MLPClassifier(max_iter = 9000)
ann_plec.fit(d_train_csv, Train_Class)

#Test the ANN model on the test molecules:
prediction_test_ann_plec_class = ann_plec.predict(d_test_csv)
prediction_test_ann_plec_prob = ann_plec.predict_proba(d_test_csv)

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_ann = pd.DataFrame({"Good_Pose_Prob": prediction_test_ann_plec_prob[:, 1],
                                "Predicted_Class": prediction_test_ann_plec_class,
                                "Real_Class": Test_Class})

rmsd = test_data.iloc[:, 1]
pose = test_data.iloc[:, 0]

plec_result_ann['RMSD'] = rmsd
plec_result_ann['Pose'] = pose

plec_result_ann.to_csv("/home/vktrannguyen/Projects/redocked/PLEC/hard_test/results/ANN/ANN_no-tuning_05.csv")